<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS)_%26_Bayesian_Personalized_Ranking_(BPR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 27.9 MB/s eta 0:00:00


In [ ]:
#!pip install -U "ray[default]"

In [ ]:
#!pip install tune-sklearn ray[tune]

In [2]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta


# Loading DataFrames



In [3]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-3-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [4]:
len(df)

2220299

In [5]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


In [6]:
len(df["BranchCustomerNbr"].unique())

17697

In [7]:
len(df["Sku"].unique())

77401

# Data Preprocessing

In [8]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)

In [9]:
df['Entry Date'] = pd.to_datetime(df['Entry Date'], format='%d.%m.%y')

In [10]:
# lines with zero shipment qty
len(df[df["Qty Shipped"] == 0])

113860

In [11]:
df[df["Qty Shipped"] == 0].head(5)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
24204,4434497,2023-01-10,NaN,2023FM01,44475054,EVERIT GMBH,Business Channel,,CD71519,1830 24G 12P CLASS4 POE-STOCK,.,490,Communications & Networking,Lan Switches Managed Layer 2,Networking,"0,00",0.0
35112,4490586,2023-02-03,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E390,TC21/TC26 HC WHITE FIVE SLOT,CHARGE ONLY CRADLE FIVE DEVICES,6026,AIDC/PoS Accessories & Supplies,Docking Station,"Other (incl. AIDC/POS, V7)","0,00",0.0
35113,4490586,2023-02-03,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E412,TC21/26 BELT CLIP HC,DISINFECTANT READY TC21/TC26 HC,5706,AIDC/PoS Barcode Scanners,Barcode Scanner Accessories,"Other (incl. AIDC/POS, V7)","0,00",0.0
40384,4437936,2023-02-17,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58182,IPHONE 14 PRO 128GB,SPACE BLACK,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40385,4437936,2023-02-17,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58183,IPHONE 14 PRO 128GB,SILVER,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0


In [12]:
len(df)

2220299

In [13]:
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"] > 0]

# dropping backlog invoices
# Specified date to filter the rows
specific_date = pd.to_datetime('2022-11-01')
# Filter the DataFrame to keep only the rows that are before or equal to the specific date
df = df[df["Entry Date"] >= specific_date]

#insert purchase indication column
df["purchase"] = 1

In [14]:
len(df)

2028956

In [15]:
len(df["BranchCustomerNbr"].unique())

13894

In [16]:
len(df["Sku"].unique())

75643

## User and sku unqiue list

In [17]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [18]:
df_2 = df[["bcn_id","sku_id", "Entry Date", "FiscalMonth", "Qty Shipped", "purchase"]].copy()

In [19]:
df_2

,bcn_id,sku_id,Entry Date,FiscalMonth,Qty Shipped,purchase
213,497,49835,2022-11-02,2022FM11,41.0,1
458,1000,52759,2022-11-03,2023FM02,140.0,1
1425,2139,53004,2022-11-08,2023FM01,1.0,1
2112,8216,33756,2022-11-09,2022FM12,5.0,1
3786,3641,35337,2022-11-14,2022FM11,1.0,1
...,...,...,...,...,...,...
2220294,11119,46629,2022-12-28,2022FM12,21.0,1
2220295,1966,24994,2023-06-06,2023FM06,1.0,1
2220296,1966,73802,2023-06-06,2023FM06,1.0,1
2220297,11681,29253,2022-12-19,2022FM12,2.0,1


In [20]:
grouped_df = df_2.groupby(['bcn_id', 'sku_id']).agg({
    'Qty Shipped': 'sum',
    'purchase': 'sum'
}).reset_index()

In [21]:
# add column that represent sku count
def sku_count(df_pl):

  df_pl['sku_count'] = df_pl.groupby('bcn_id')['sku_id'].transform('nunique')

  return df_pl

In [22]:
grouped_df = sku_count(grouped_df)
grouped_df.head(4)

,bcn_id,sku_id,Qty Shipped,purchase,sku_count
0,0,928,1.0,1,99
1,0,3422,1.0,1,99
2,0,3423,4.0,2,99
3,0,3424,3.0,1,99


### Threshold weight (QTY / purchase)



1.   Sku Count >= 2
2.   Sku Count >= 2 & Purchase > 2



In [32]:
data_1 = grouped_df[grouped_df.sku_count >= 2]
data_1

,bcn_id,sku_id,Qty Shipped,purchase,sku_count
0,0,928,1.0,1,99
1,0,3422,1.0,1,99
2,0,3423,4.0,2,99
3,0,3424,3.0,1,99
4,0,4189,4.0,1,99
...,...,...,...,...,...
544411,13891,33754,9.0,1,3
544412,13891,42310,2.0,1,3
544413,13891,45247,2.0,1,3
544415,13893,36049,25.0,1,2


In [23]:
data_2 = grouped_df[(grouped_df.sku_count >= 2) & (grouped_df.purchase > 3)]
data_2

,bcn_id,sku_id,Qty Shipped,purchase,sku_count
27,0,14912,18.0,7,99
79,0,45256,11.0,6,99
80,0,45260,13.0,4,99
164,3,25971,248.0,16,20
167,3,30625,70.0,5,20
...,...,...,...,...,...
544223,13843,2402,101.0,5,45
544249,13843,42597,285.0,6,45
544255,13843,45721,47.0,4,45
544256,13843,46704,110.0,5,45


In [33]:
data_bin = grouped_df[(grouped_df.sku_count >= 2)]
data_bin["purchase"] = 1
data_bin

<ipython-input-33-60f0fa2ec5ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bin["purchase"] = 1


,bcn_id,sku_id,Qty Shipped,purchase,sku_count
0,0,928,1.0,1,99
1,0,3422,1.0,1,99
2,0,3423,4.0,1,99
3,0,3424,3.0,1,99
4,0,4189,4.0,1,99
...,...,...,...,...,...
544411,13891,33754,9.0,1,3
544412,13891,42310,2.0,1,3
544413,13891,45247,2.0,1,3
544415,13893,36049,25.0,1,2


In [152]:
# change to binary data for BPR
bpr_data = data_2[["bcn_id","sku_id"]]
bpr_data["purchase"] = 1
bpr_data.head(2)

<ipython-input-152-7cae61c6fbf9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bpr_data["purchase"] = 1


,bcn_id,sku_id,purchase
27,0,14912,1
79,0,45256,1


In [155]:
bpr_data

,bcn_id,sku_id,purchase
27,0,14912,1
79,0,45256,1
80,0,45260,1
164,3,25971,1
167,3,30625,1
...,...,...,...
544223,13843,2402,1
544249,13843,42597,1
544255,13843,45721,1
544256,13843,46704,1


## LOOCV

with K=1 so 1 item per user gets dropped out

In [159]:
def train_test_split(df_pl, holdout_num):
    """ perform training testing split

    @param df: dataframe
    @param holdhout_num: number of items to be held out per user as testing items

    @return df_train: training data
    @return df_test testing data

    """
    # perform deep copy to avoid modification on the original dataframe
    df_train = df_pl.copy(deep=True)
    df_test = df_pl.copy(deep=True)

    # get test set
    df_test = df_test.groupby(['bcn_id']).head(holdout_num).reset_index()

    # get train set
    df_train = df_train.merge(
        df_test[['bcn_id', 'sku_id']].assign(remove=1),
        how='left'
    ).query('remove != 1').drop('remove', 1).reset_index(drop=True)

    # drop index
    df_test = df_test.drop(columns=["index"])

    # sanity check to make sure we're not duplicating/losing data
    assert len(df_pl) == len(df_train) + len(df_test)

    return df_train, df_test

In [34]:
df_qty_train_1, df_qty_test_1 = train_test_split(data_1[["bcn_id", "sku_id", "Qty Shipped"]], 1)
df_frequency_train_1, df_frequency_test_1 = train_test_split(data_1[["bcn_id", "sku_id", "purchase"]], 1)
df_bin_train, df_bin_test = train_test_split(data_bin[["bcn_id", "sku_id", "purchase"]], 1)

<ipython-input-29-405fbbb36640>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)
<ipython-input-29-405fbbb36640>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)
<ipython-input-29-405fbbb36640>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


In [35]:
df_qty_train_2, df_qty_test_2 = train_test_split(data_2[["bcn_id", "sku_id", "Qty Shipped"]], 1)
df_frequency_train_2, df_frequency_test_2 = train_test_split(data_2[["bcn_id", "sku_id", "purchase"]], 1)

<ipython-input-29-405fbbb36640>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)
<ipython-input-29-405fbbb36640>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


In [160]:
bpr_data_train, bpr_data_test = train_test_split(bpr_data, 1)

<ipython-input-159-405fbbb36640>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


In [161]:
len(bpr_data_test)

3626

### Stratifying for same bcns in train test

In [41]:
def sku_bcn_check(train_df, test_df):

  train_bcn_ids = set(train_df.bcn_id.unique())
  test_bcn_ids = set(test_df.bcn_id.unique())

  train_sku_ids = set(df_qty_train_2.sku_id.unique())
  test_sku_ids = set(df_qty_test_2.sku_id.unique())

  return test_bcn_ids.issubset(train_bcn_ids), test_sku_ids.issubset(train_sku_ids)

In [164]:
print(sku_bcn_check(df_qty_train_1, df_qty_test_1), sku_bcn_check(df_qty_train_2, df_qty_test_2), sku_bcn_check(df_frequency_train_1,df_frequency_test_1),
      sku_bcn_check(df_frequency_train_2,df_frequency_test_2), sku_bcn_check(df_bin_train, df_bin_train), sku_bcn_check(bpr_data_train, bpr_data_test))

(True, False) (False, False) (True, False) (False, False) (True, False) (False, False)


In [163]:
def stratify_bcn(train_df, test_df):
  """ remove users from test set, that are not in train
    @params: train and test dfs

    @return stratified_df_test testing data
  """
  # Get the unique 'bcn_id' values from df_train
  trained_bcn_ids = train_df['bcn_id'].unique()
  # Filter out the rows in df_test that have 'bcn_id' values not present in trained_bcn_ids
  stratified_df_test = test_df[test_df['bcn_id'].isin(trained_bcn_ids)]

  return stratified_df_test

In [165]:
stratified_df_qty_test_2 = stratify_bcn(df_qty_train_2, df_qty_test_2)
stratified_df_frequency_test_2 = stratify_bcn(df_frequency_train_2, df_frequency_test_2)
stratified_bpr_data_test = stratify_bcn(bpr_data_train, bpr_data_test)

In [166]:
print(sku_bcn_check(df_qty_train_1, df_qty_test_1), sku_bcn_check(df_qty_train_2, stratified_df_qty_test_2), sku_bcn_check(df_frequency_train_1,df_frequency_test_1),
      sku_bcn_check(df_frequency_train_2,stratified_df_frequency_test_2), sku_bcn_check(df_bin_train, df_bin_test))

(True, False) (True, False) (True, False) (True, False) (True, False)


### Negative sampling


In [ ]:
def negative_sampling(df_train, bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param sku_ids: list of sku ids
    @param items: unique list of sku ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'purchase'])

    #df_train = df_train[['bcn_id', 'sku_id']].assign(purchase=1)
    df_train = pd.concat([df_train, df_neg], ignore_index=True)

    return df_train

In [ ]:
def negative_sampling_2(df_train, bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param sku_ids: list of sku ids
    @param items: unique list of sku ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'Qty Shipped'])

    #df_train = df_train[['bcn_id', 'sku_id']].assign(purchase=1)
    df_train = pd.concat([df_train, df_neg], ignore_index=True)

    return df_train

In [ ]:
# create final training
neg_df_qty_train_1 = negative_sampling_2(
    df_train = df_qty_train_1,
    bcn_ids = df_qty_train_1.bcn_id.unique(),
    sku_ids = df_qty_train_1.sku_id.unique(),
    items = df.sku_id.unique(),
    n_neg = 10
)
neg_df_qty_train_2 = negative_sampling_2(
    df_train = df_qty_train_2,
    bcn_ids = df_qty_train_2.bcn_id.unique(),
    sku_ids = df_qty_train_2.sku_id.unique(),
    items = df.sku_id.unique(),
    n_neg = 10
)

In [ ]:
# create final training
neg_df_frequency_train_1 = negative_sampling(
    df_train = df_frequency_train_1,
    bcn_ids = df_frequency_train_1.bcn_id.unique(),
    sku_ids = df_frequency_train_1.sku_id.unique(),
    items = df.sku_id.unique(),
    n_neg = 10

)
neg_df_frequency_train_2 = negative_sampling(
    df_train = df_frequency_train_2,
    bcn_ids = df_frequency_train_2.bcn_id.unique(),
    sku_ids = df_frequency_train_2.sku_id.unique(),
    items = df.sku_id.unique(),
    n_neg = 10
)

In [ ]:
neg_df_bin_train = negative_sampling(
    df_train = df_bin_train,
    bcn_ids = df_bin_train.bcn_id.unique(),
    sku_ids = df_bin_train.sku_id.unique(),
    items = df.sku_id.unique(),
    n_neg = 10
)

## Create sparse user x item and item x user matrices

In [ ]:
def create_qty_user_item_csr_matrix(df_pl_train, df_pl_test):
  """ create a sparse data matrix with scipy sparse matrix
      @return sparse user x item matrix
  """
  sparse_user_item_train = sparse.csr_matrix((df_pl_train['Qty Shipped'], (df_pl_train['bcn_id'], df_pl_train['sku_id'])))
  sparse_user_item_test = sparse.csr_matrix((df_pl_test['Qty Shipped'], (df_pl_test['bcn_id'], df_pl_test['sku_id'])))

  return sparse_user_item_train, sparse_user_item_test

In [167]:
def create_user_item_csr_matrix(df_pl_train, df_pl_test):
  """ create a sparse data matrix with scipy sparse matrix
      @return sparse user x item matrix
  """
  sparse_user_item_train = sparse.csr_matrix((df_pl_train['purchase'].astype(float), (df_pl_train['bcn_id'], df_pl_train['sku_id'])))
  sparse_user_item_test = sparse.csr_matrix((df_pl_test['purchase'].astype(float), (df_pl_test['bcn_id'], df_pl_test['sku_id'])))

  return sparse_user_item_train, sparse_user_item_test

In [ ]:
def create_item_user_csr_matrix(df_pl):
  """ create a sparse data matrix with scipy sparse matrix
      @return sparse user x item matrix
  """
  sparse_item_user = sparse.csr_matrix((df_pl['purchase'].astype(float), (df_pl['sku_id'], df_pl['bcn_id'])))

  return sparse_item_user

In [ ]:
# only positive sparse matrices
sparse_user_item_qty_train_1, sparse_user_item_qty_test_1 = create_qty_user_item_csr_matrix(df_qty_train_1, df_qty_test_1)
sparse_user_item_qty_train_2, sparse_user_item_qty_test_2 = create_qty_user_item_csr_matrix(df_qty_train_2, stratified_df_qty_test_2)

sparse_user_item_frequency_train_1, sparse_user_item_frequency_test_1 = create_user_item_csr_matrix(df_frequency_train_1, df_frequency_test_1)
sparse_user_item_frequency_train_2, sparse_user_item_frequency_test_2 = create_user_item_csr_matrix(df_frequency_train_2, stratified_df_frequency_test_2)

sparse_user_item_bin_train, sparse_user_item_bin_test = create_user_item_csr_matrix(df_bin_train, df_bin_test)

In [ ]:
# negative sampled sparse matrices
neg_sparse_user_item_qty_train_1, neg_sparse_user_item_qty_test_1 = create_qty_user_item_csr_matrix(neg_df_qty_train_1, df_qty_test_1)
neg_sparse_user_item_qty_train_2, neg_sparse_user_item_qty_test_2 = create_qty_user_item_csr_matrix(neg_df_qty_train_2, stratified_df_qty_test_2)

neg_sparse_user_item_frequency_train_1, neg_sparse_user_item_frequency_test_1 = create_user_item_csr_matrix(neg_df_frequency_train_1, df_frequency_test_1)
neg_sparse_user_item_frequency_train_2, neg_sparse_user_item_frequency_test_2 = create_user_item_csr_matrix(neg_df_frequency_train_2, stratified_df_frequency_test_2)

In [ ]:
sparse_user_item_bin_test

<13894x75631 sparse matrix of type '<class 'numpy.float64'>'
	with 11328 stored elements in Compressed Sparse Row format>

##Create unique sku and user lists

In [44]:
skus = df[["sku_id", "Sku" , "Product Descr1", "Product Descr2", "ProductGroupDescription"]].drop_duplicates()

In [45]:
users = df[["bcn_id", "BranchCustomerNbr"]].drop_duplicates()

# **ALS Model**

ALS hyperparameters are:


*   factors: The number of latent factors to compute
*   regularization: The regularization factor to use
*   alpha: The weight to give to positive examples
*   iterations: The number of ALS iterations to use when fitting data





## ALS Model param tuning

In [162]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k
from sklearn.model_selection import GridSearchCV

In [ ]:
from IPython.utils.sysinfo import num_cpus
# Define the hyperparameter search space
factors_list = [100, 150, 200]
iterations_list = [10, 20]
regularization_list = [0.01, 0.1]
alpha_list = [5, 20, 40]

best_precision = 0
best_factors = None
best_iterations = None
best_regularization = None
best_alpha = None

# Perform manual grid search
for factors in factors_list:
    for iterations in iterations_list:
        for regularization in regularization_list:
            for alpha in alpha_list:
                # Create the ALS model with the current hyperparameters
                als_model = AlternatingLeastSquares(factors=factors, iterations=iterations,
                                                    regularization=regularization, alpha=alpha)
                als_model.fit(neg_sparse_user_item_qty_train_2, show_progress=False)

                # Evaluate the model on the test set
                precision = precision_at_k(als_model, neg_sparse_user_item_qty_train_2, neg_sparse_user_item_qty_test_2, K=10, num_threads=10, show_progress=False)

                # Check if current hyperparameters give better precision
                if precision > best_precision:
                    best_precision = precision
                    best_factors = factors
                    best_iterations = iterations
                    best_regularization = regularization
                    best_alpha = alpha

print(f"Best Factors: {best_factors}")
print(f"Best Iterations: {best_iterations}")
print(f"Best Regularization: {best_regularization}")
print(f"Best Alpha: {best_alpha}")
print(f"Precision@10 on Test Set: {best_precision}")

Best Factors: 100
Best Iterations: 20
Best Regularization: 0.01
Best Alpha: 5
Precision@10 on Test Set: 0.21248025276461296


*Qty based model:*



1.   Best Factors: 100
2.   Best Iterations: 20
3.   Best Regularization: 0.01
4.   Best alpha: 5

Precision@10 on Test Set: 0.21248025276461296

In [ ]:
from IPython.utils.sysinfo import num_cpus
# Define the hyperparameter search space
factors_list = [100, 150, 200]
iterations_list = [10, 20]
regularization_list = [0.01, 0.1]
alpha_list = [5, 20, 40]

best_precision = 0
best_factors = None
best_iterations = None
best_regularization = None
best_alpha = None

# Perform manual grid search
for factors in factors_list:
    for iterations in iterations_list:
        for regularization in regularization_list:
            for alpha in alpha_list:
                # Create the ALS model with the current hyperparameters
                als_model = AlternatingLeastSquares(factors=factors, iterations=iterations,
                                                    regularization=regularization, alpha=alpha)
                als_model.fit(neg_sparse_user_item_frequency_train_2, show_progress=False)

                # Evaluate the model on the test set
                precision = precision_at_k(als_model, neg_sparse_user_item_frequency_train_2, neg_sparse_user_item_frequency_test_2, K=10, num_threads=10, show_progress=False)

                # Check if current hyperparameters give better precision
                if precision > best_precision:
                    best_precision = precision
                    best_factors = factors
                    best_iterations = iterations
                    best_regularization = regularization
                    best_alpha = alpha

print(f"Best Factors: {best_factors}")
print(f"Best Iterations: {best_iterations}")
print(f"Best Regularization: {best_regularization}")
print(f"Best Alpha: {best_alpha}")
print(f"Precision@10 on Test Set: {best_precision}")

Best Factors: 100
Best Iterations: 10
Best Regularization: 0.01
Best Alpha: 5
Precision@10 on Test Set: 0.24328593996840442


*Frequency model*:


1.   Best Factors: 100
2.   Best Iterations: 10
3.   Best Regularization: 0.01
4.   Best Alpha: 5

Precision@10 on Test Set: 0.24328593996840442

In [ ]:
# def validate(df_train, df_test, factors=200, iterations=20, regularization=0.01, alpha=1, show_progress=True):
#     """ Train an ALS model with <<factors>> (embeddings dimension)
#     for <<iterations>> over matrices and validate with MAP@12
#     """
#     model = implicit.als.AlternatingLeastSquares(factors=factors,
#                                                  iterations=iterations,
#                                                  regularization=regularization,
#                                                  alpha = alpha,
#                                                  random_state=42)
#     model.fit(df_train, show_progress=show_progress)

#     # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
#     # TODO: change MAP@12 to a library that allows repeated items in prediction
#     prec10 = precision_at_k(model, df_train, df_test, K=10, show_progress=show_progress, num_threads=4)
#     print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} - Alpha: {alpha:>1} ==> PREC@10: {prec10:6.5f}")
#     return prec10

In [ ]:
# %%time
# best_prec10 = 0
# for factors in [50, 100, 150, 200]:
#     for iterations in [5, 10, 20]:
#         for regularization in [0.01, 0.1]:
#           for alpha in [5, 20, 40]:
#             prec10 = validate(sparse_user_item_qty_train_2, sparse_user_item_qty_test_2, factors, iterations, regularization, show_progress=False)
#             if prec10 > best_prec10:
#                 best_prec10 = prec10
#                 best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization, 'alpha':alpha}
#                 print(f"Best MAP@12 found. Updating: {best_params}")

In [ ]:
#test if model works on data
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=10)
model.fit(neg_sparse_user_item_qty_train_2)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 8.16 s, sys: 11.2 s, total: 19.4 s
Wall time: 13.1 s


### ALS QTY Based

In [ ]:
%%time
model_2 = implicit.als.AlternatingLeastSquares(factors=100, alpha=5, regularization=0.01, iterations=10)
model_2.fit(neg_sparse_user_item_qty_train_2)
prec10 = precision_at_k(model_2, neg_sparse_user_item_qty_train_2, neg_sparse_user_item_qty_test_2, K=10, num_threads=10)
auc_10 = AUC_at_k(model_2, neg_sparse_user_item_qty_train_2, neg_sparse_user_item_qty_test_2, K=10, num_threads=10)
print(prec10, auc_10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2532 [00:00<?, ?it/s]

  0%|          | 0/2532 [00:00<?, ?it/s]

0.19036334913112166 0.5951209375533248
CPU times: user 21.7 s, sys: 19.1 s, total: 40.7 s
Wall time: 38.5 s


In [ ]:
%%time
model_2 = implicit.als.AlternatingLeastSquares(factors=150, alpha=5, regularization=0.01, iterations=20)
model_2.fit(neg_sparse_user_item_qty_train_2)
prec10 = precision_at_k(model_2, neg_sparse_user_item_qty_train_2, neg_sparse_user_item_qty_test_2, K=10, num_threads=10)
auc_10 = AUC_at_k(model_2, neg_sparse_user_item_qty_train_2, neg_sparse_user_item_qty_test_2, K=10, num_threads=10)
print(prec10, auc_10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/2532 [00:00<?, ?it/s]

  0%|          | 0/2532 [00:00<?, ?it/s]

0.21248025276461296 0.6061800947031165
CPU times: user 54.2 s, sys: 48.1 s, total: 1min 42s
Wall time: 1min 6s


## ALS Best Model Performance

In [ ]:
precision_als = precision_at_k(model_2, neg_sparse_user_item_qty_train_2, neg_sparse_user_item_qty_test_2, K=10, num_threads=10)
precision_als

  0%|          | 0/2532 [00:00<?, ?it/s]

0.21011058451816747

In [ ]:
map_precision_als = mean_average_precision_at_k(model_2, neg_sparse_user_item_qty_train_2, neg_sparse_user_item_qty_test_2, K=10, num_threads=10)
precision_als

  0%|          | 0/2532 [00:00<?, ?it/s]

0.21011058451816747

In [ ]:
ndcg_10 = ndcg_at_k(model_2, neg_sparse_user_item_qty_train_2, neg_sparse_user_item_qty_test_2, K=10, num_threads=10)
ndcg_10

  0%|          | 0/2532 [00:00<?, ?it/s]

0.12725223548888098

### Frequency Based ALS

In [ ]:
%%time
model_4 = implicit.als.AlternatingLeastSquares(factors = 100, iterations = 10, regularization = 0.01, alpha = 5)
model_4.fit(neg_sparse_user_item_frequency_train_2)
prec10 = precision_at_k(model_4, neg_sparse_user_item_frequency_train_2, neg_sparse_user_item_frequency_test_2, K=10, num_threads=10)
auc_10 = AUC_at_k(model_4, neg_sparse_user_item_frequency_train_2, neg_sparse_user_item_frequency_test_2, K=10, num_threads=10)
print(prec10, auc_10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2532 [00:00<?, ?it/s]

  0%|          | 0/2532 [00:00<?, ?it/s]

0.24052132701421802 0.620201405081924
CPU times: user 22 s, sys: 20.2 s, total: 42.3 s
Wall time: 26.5 s


## ALS Best Model Performance

In [ ]:
precision_als = precision_at_k(model_4, neg_sparse_user_item_frequency_train_2, neg_sparse_user_item_frequency_test_2, K=10, num_threads=10)
precision_als

  0%|          | 0/2532 [00:00<?, ?it/s]

0.24052132701421802

In [ ]:
map_precision_als = mean_average_precision_at_k(model_4, neg_sparse_user_item_frequency_train_2, neg_sparse_user_item_frequency_test_2, K=10, num_threads=10)
precision_als

  0%|          | 0/2532 [00:00<?, ?it/s]

0.24052132701421802

In [ ]:
nendcg_10 = ndcg_at_k(model_4, neg_sparse_user_item_frequency_train_2, neg_sparse_user_item_frequency_test_2, K=10, num_threads=10)
ndcg_10

  0%|          | 0/2532 [00:00<?, ?it/s]

0.12725223548888098

## ALS Best Model Recommendations

In [ ]:
user_skucounts = data_1[["bcn_id", "sku_count"]].drop_duplicates()
users_less_20 = user_skucounts[user_skucounts.sku_count < 20]
users_less_20 = users_less_20["bcn_id"].values
users_more_20 = user_skucounts[user_skucounts.sku_count >= 20]
users_more_20 = users_more_20["bcn_id"].values

2970

In [79]:
 # Assuming you want recommendations for user with ID 10
import random
user_id = random.choice(users_less_20)
user_id_2 = random.choice(users_more_20)
print(user_id)
print(user_id_2)

NameError: ignored

In [ ]:
df[df.bcn_id==6676]

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase,bcn_id,sku_id
205630,4458076,2023-06-13,13.06.2023 16:05:07,2023FM06,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,J154510,MINI DP TO DVI ADAPTER BLACK,MINI DP TO DVI-D ADPTR 1080P FHD,1230,Cables,Monitor Cables & Adapters,Printers & Peripherals,"30,84",6.0,1,6676,66008
231213,4441851,2023-06-09,09.06.2023 10:08:02,2023FM06,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,2M35066,550 - SHEET PAPER TRAY,M454/ M479,794,Printers/AIO/Copiers/Fax,Printer/Plotter Accs,Printers & Peripherals,"91,70",1.0,1,6676,9804
274666,4416386,2023-06-02,02.06.2023 11:58:03,2023FM06,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,54134C4,WORRY FREE 5 SERVICES,RNW LIZ 24 M - 0006 - 0010 USER,1569,Softwares,Other Security Software,Software,"433,10",10.0,1,6676,16282
628430,4443244,2023-04-18,18.04.2023 09:49:54,2023FM04,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,Z698640,BU+ESS UNIV LIC COM,SUB UPF BILL 3Y 05 INS,1563,Softwares,Back-Up Software,Software,"1.027,72",1.0,1,6676,75516
730504,4404020,2023-04-04,04.04.2023 16:56:54,2023FM04,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,2P4W741,BU EXEC V-RAY ED,COR RNW MNT ESS 1Y,1569,Softwares,Other Security Software,Software,"445,56",2.0,1,6676,11783
991490,4487753,2023-03-07,07.03.2023 13:43:51,2023FM03,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,Z698640,BU+ESS UNIV LIC COM,SUB UPF BILL 3Y 05 INS,1563,Softwares,Back-Up Software,Software,"1.027,71",1.0,1,6676,75516
1103121,4434625,2023-02-22,22.02.2023 11:23:37,2023FM02,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,CG17308A,PULSE AMD RADEON RX7900 XTX,GAM. OC 24GB GDDR6 DUAL HDMI/DP,898,Multimedia & Games,Video Card,CE,"844,00",1.0,1,6676,55956
1595761,4400705,2022-12-29,29.12.2022 09:10:07,2022FM12,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,2P4W770,BU EXEC V-RAY ED,COR NEW BDL LIC ESS MNT 1Y,1563,Softwares,Back-Up Software,Software,"850,96",1.0,1,6676,11787
1682133,4474012,2022-12-20,20.12.2022 13:12:42,2022FM12,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,2P4W741,BU EXEC V-RAY ED,COR RNW MNT ESS 1Y,1569,Softwares,Other Security Software,Software,"222,78",1.0,1,6676,11783
1720648,4459442,2022-12-16,16.12.2022 13:35:58,2022FM12,44505829,CNS-IT GMBH SYSTEMHAUS FUER,SMB Channel,PCSM,2P4W741,BU EXEC V-RAY ED,COR RNW MNT ESS 1Y,1569,Softwares,Other Security Software,Software,"222,78",1.0,1,6676,11783


In [ ]:
df[df.bcn_id==5621]

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase,bcn_id,sku_id
98408,4412286,2023-06-28,28.06.2023 09:26:15,2023FM06,44486747,HAMM-INDUSTRIES GMBH & CO. KG,SMB Channel,,CE64969,ACROBAT PRO 2020 TLP COM,UPG,1509,Softwares,Desktop Publishing Software,Software,"274,61",1.0,1,5621,44361
157223,4480059,2023-06-19,19.06.2023 16:42:32,2023FM06,44486747,HAMM-INDUSTRIES GMBH & CO. KG,SMB Channel,,CE64969,ACROBAT PRO 2020 TLP COM,UPG,1509,Softwares,Desktop Publishing Software,Software,"274,61",1.0,1,5621,44361
157236,4480041,2023-06-19,19.06.2023 16:39:55,2023FM06,44486747,HAMM-INDUSTRIES GMBH & CO. KG,SMB Channel,,CE64969,ACROBAT PRO 2020 TLP COM,UPG,1509,Softwares,Desktop Publishing Software,Software,"274,62",1.0,1,5621,44361
172904,4476508,2023-06-19,19.06.2023 10:24:03,2023FM06,44486747,HAMM-INDUSTRIES GMBH & CO. KG,SMB Channel,,5663421,SP-1120N,A4 DUPLEX OFFICE SCANNER,1701,Scanners and cameras,Scanner Din A4,Printers & Peripherals,"12.638,00",71.0,1,5621,17605
172905,4476508,2023-06-19,19.06.2023 10:24:03,2023FM06,44486747,HAMM-INDUSTRIES GMBH & CO. KG,SMB Channel,,5663422,SP-1130N,A4 DUPLEX OFFICE SCANNER,1701,Scanners and cameras,Scanner Din A4,Printers & Peripherals,"6.006,15",27.0,1,5621,17606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008350,4420270,2022-11-21,22.11.2022 07:50:14,2022FM11,44486747,HAMM-INDUSTRIES GMBH & CO. KG,SMB Channel,,L05K648,STORAGE WAREHOUSING VAS,EXT. WH - HAMM,2006,Logistics Services,Ingram Specific Configuation Fees,"Other (incl. AIDC/POS, V7)","141,44",64.0,1,5621,66356
2119739,4469225,2022-11-11,11.11.2022 10:43:11,2022FM11,44486747,HAMM-INDUSTRIES GMBH & CO. KG,SMB Channel,,5662915,FI-7280 DOCUMENT SCANNER,IN,1701,Scanners and cameras,Scanner Din A4,Printers & Peripherals,"1.384,52",1.0,1,5621,17537
2119740,4469225,2022-11-11,11.11.2022 10:43:11,2022FM11,44486747,HAMM-INDUSTRIES GMBH & CO. KG,SMB Channel,,5663203,SP-1425 SCANNER,IN,1701,Scanners and cameras,Scanner Din A4,Printers & Peripherals,"251,00",1.0,1,5621,17556
2161508,4443508,2022-11-07,07.11.2022 16:22:47,2022FM11,44486747,HAMM-INDUSTRIES GMBH & CO. KG,SMB Channel,,CE64969,ACROBAT PRO 2020 TLP COM,UPG,1509,Softwares,Desktop Publishing Software,Software,"274,62",1.0,1,5621,44361


In [ ]:
# Now you can call the recommend function
recommended = model_3.recommend(user_id, user_items[user_id], N=100, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab.sort_values(by="conf", ascending=False)

NameError: ignored

In [ ]:
user_purchases = pd.DataFrame(df[df.bcn_id == user_id][["Product Descr1","Sku", "ProductGroupDescription", "purchase"]].groupby(
                              ["Sku","Product Descr1", "ProductGroupDescription"])["purchase"].sum().sort_values(ascending=False))
user_purchases[0:20]

# BPR Model

In [47]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808329 sha256=12108e816ff57ec1cfc7dd81c88ada80d3fc5d63c35f183291e0d109b117da4a
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [227]:
from implicit.bpr import BayesianPersonalizedRanking as BPR

In [169]:
len(bpr_data_train)

127033

In [172]:
len(bpr_data_test)

3626

In [173]:
bpr_data_train_2 = bpr_data_train.copy()

In [174]:
# Step 1: Create a dictionary to store interacted SKUs for each bcn_id
interacted_skus = {}

# Step 2: Populate the dictionary
for _, row in bpr_data_train_2.iterrows():
    bcn_id = row['bcn_id']
    sku_id = row['sku_id']
    if bcn_id not in interacted_skus:
        interacted_skus[bcn_id] = {sku_id}
    else:
        interacted_skus[bcn_id].add(sku_id)

# Step 3 and 4: Sample one SKU that each bcn_id did not interact with and add as new rows
new_rows = []
for bcn_id, sku_set in interacted_skus.items():
    all_skus = set(df['sku_id'].unique())  # List of all unique SKUs
    non_interacted_skus = all_skus - sku_set  # SKUs not interacted by the bcn_id
    if non_interacted_skus:
        new_sku_id = random.choice(list(non_interacted_skus))
        new_rows.append({'bcn_id': bcn_id, 'sku_id': new_sku_id, 'purchase': 0})

# Step 5: Add new rows to the existing DataFrame
bpr_data_train_2 = bpr_data_train_2.append(new_rows, ignore_index=True)

# (Optional) Reset the index if needed
bpr_data_train_2.reset_index(drop=True, inplace=True)

bpr_data_train_2.head()

KeyboardInterrupt: ignored

In [90]:
# Step 1: Create a dictionary to store interacted SKUs for each bcn_id
interacted_skus = {}

# Step 2: Populate the dictionary
for _, row in bpr_data_train_2.iterrows():
    bcn_id = row['bcn_id']
    sku_id = row['sku_id']
    if bcn_id not in interacted_skus:
        interacted_skus[bcn_id] = {sku_id}
    else:
        interacted_skus[bcn_id].add(sku_id)

# Step 3 and 4: Sample dynamic number of new rows for each bcn_id with not-purchased SKUs
new_rows = []

for bcn_id, sku_set in interacted_skus.items():
    all_skus = set(df['sku_id'].unique())  # List of all unique SKUs
    non_interacted_skus = all_skus - sku_set  # SKUs not interacted by the bcn_id
    num_samples_per_bcn = min(len(non_interacted_skus), len(sku_set))  # Dynamic number of samples

    if num_samples_per_bcn > 0:
        sampled_skus = random.sample(non_interacted_skus, num_samples_per_bcn)
        for new_sku_id in sampled_skus:
            new_rows.append({'bcn_id': bcn_id, 'sku_id': new_sku_id, 'purchase': 0})

# Step 5: Add new rows to the existing DataFrame
bpr_data_train_2 = bpr_data_train_2.append(new_rows, ignore_index=True)

# (Optional) Reset the index if needed
bpr_data_train_2.reset_index(drop=True, inplace=True)

bpr_data_train_2

<ipython-input-90-ecdcbb04cf7b>:22: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sampled_skus = random.sample(non_interacted_skus, num_samples_per_bcn)
<ipython-input-90-ecdcbb04cf7b>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bpr_data_train_2 = bpr_data_train_2.append(new_rows, ignore_index=True)


,bcn_id,sku_id,purchase
0,0,45256,1
1,0,45260,1
2,3,30625,1
3,3,30628,1
4,3,42771,1
...,...,...,...
254061,13720,14213,0
254062,13727,16050,0
254063,13843,62063,0
254064,13843,46496,0


In [91]:
len(bpr_data_train) * 2 == len(bpr_data_train_2)

True

In [93]:
bpr_data_train_2[bpr_data_train_2.bcn_id == 0]

,bcn_id,sku_id,purchase
0,0,45256,1
1,0,45260,1
127033,0,11353,0
127034,0,53311,0


In [113]:
# create sparse matrix (bpr_data_train, bpr_data_test)
bpr_csr_data_train, bpr_csr_data_test = create_user_item_csr_matrix(bpr_data_train_2, stratified_bpr_data_test)

## Implicit BPR

In [114]:
model_bpr = BPR(factors=100, iterations=10, learning_rate=0.01, regularization=0.01)

In [115]:
model_bpr.fit(bpr_csr_data_train)

  0%|          | 0/10 [00:00<?, ?it/s]

In [116]:
auc = AUC_at_k(model_bpr, bpr_csr_data_train, bpr_csr_data_test, K=10, num_threads=10, show_progress=True)
auc

  0%|          | 0/2532 [00:00<?, ?it/s]

0.5030934183210305

In [118]:
prec10 = precision_at_k(model_bpr, bpr_csr_data_train, bpr_csr_data_test, K=10, num_threads=10, show_progress=True)
prec10

  0%|          | 0/2532 [00:00<?, ?it/s]

0.00631911532385466

In [98]:
map10 = mean_average_precision_at_k(model_bpr, bpr_csr_data_train, bpr_csr_data_test, K=10, show_progress=True,
                                    num_threads=10)
map10

  0%|          | 0/2532 [00:00<?, ?it/s]

0.01240408485669149

In [100]:
ndcg10 = ndcg_at_k(model_bpr, bpr_csr_data_train, bpr_csr_data_test, K=10, show_progress=True,
                                    num_threads=10)
ndcg10

  0%|          | 0/2532 [00:00<?, ?it/s]

0.023667963170132894

In [ ]:
 # Assuming you want recommendations for user with ID 10
user_id = 275

# Now you can call the recommend function
recommended = model_bpr.recommend(user_id, user_items[user_id], N=10, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab.sort_values(by="conf", ascending=False)

## LightFM BPR

In [228]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score
from lightfm.datasets import fetch_movielens

In [175]:
bpr_data

,bcn_id,sku_id,purchase
27,0,14912,1
79,0,45256,1
80,0,45260,1
164,3,25971,1
167,3,30625,1
...,...,...,...
544223,13843,2402,1
544249,13843,42597,1
544255,13843,45721,1
544256,13843,46704,1


In [176]:
# Extract 'bcn_id' and 'sku_id' columns from df1
bcn_bu = df[["bcn_id", "BusinessUnitLevel2"]].drop_duplicates()
sku_group = df[["sku_id", "ProductGroupMasterDescription"]].drop_duplicates()

In [179]:
# Map 'BusinessUnitLevel2' and 'ProductGroupMasterDescription' in df2 based on 'bcn_id' and 'sku_id' mapping
# bpr_data_2 = bpr_data.merge(bcn_bu, on="bcn_id", how="left")
# bpr_data_2 = bpr_data_2.merge(sku_group, on="sku_id", how="left")
# bpr_data = bpr_data[["bcn_id", "sku_id", "purchase", "BusinessUnitLevel2_x", "ProductGroupMasterDescription_x"]]
bpr_data_2.columns

Index(['bcn_id', 'sku_id', 'purchase', 'BusinessUnitLevel2',
       'ProductGroupMasterDescription'],
      dtype='object')

In [194]:
bpr_data_2.head()

,bcn_id,sku_id,purchase,BusinessUnitLevel2,ProductGroupMasterDescription
0,0,14912,1,Export Channel (DE),Communications & Networking
1,0,45256,1,Export Channel (DE),Computer Systems
2,0,45260,1,Export Channel (DE),Computer Systems
3,3,25971,1,Export Channel (DE),Memory and Processors
4,3,30625,1,Export Channel (DE),Memory and Processors


In [186]:
user_ids = bpr_data_2.bcn_id.unique()
sku_ids = bpr_data_2.sku_id.unique()

In [193]:
len(sku_ids)

18553

In [198]:
dataset = Dataset()
dataset.fit(user_ids, sku_ids, user_features=bpr_data_2["BusinessUnitLevel2"].unique(),
            item_features=bpr_data_2["ProductGroupMasterDescription"].unique())

In [207]:
# Step 3: Create a dictionary to represent item features
item_features_dict = {}

# For item features (ProductGroupMasterDescription)
for _, row in bpr_data_2.iterrows():
    item_id = row['sku_id']
    feature_names = [str(row['ProductGroupMasterDescription'])]
    item_features_dict[item_id] = feature_names

In [208]:
# Step 3: Create a dictionary to represent item features
customer_features_dict = {}

# For customer features (ProductGroupMasterDescription)
for _, row in bpr_data_2.iterrows():
    cust_id = row['bcn_id']
    feature_names = [str(row['BusinessUnitLevel2'])]
    customer_features_dict[cust_id] = feature_names

In [222]:
#dataset.build_item_features(item_features_dict)
#dataset.build_user_features(customer_features_dict)
item_features = dataset.build_item_features(item_features_dict.items())
user_features = dataset.build_user_features(customer_features_dict.items())

In [216]:
interactions, weights = dataset.build_interactions(zip(user_ids, sku_ids))

In [226]:
intera

AttributeError: ignored

In [223]:
model = LightFM(loss='bpr')
model.fit(interactions, item_features=item_features, user_features=user_features)

## BPR Model param tuning